# Name: Siem Hagos ID: 20086
### Course: CS570

# Introducing Deep Learning Pipelines for Apache Spark

Deep Learning Pipelines is a new library published by Databricks to provide high-level APIs for scalable deep learning model application and transfer learning via integration of popular deep learning libraries with MLlib Pipelines and Spark SQL. For an overview and the philosophy behind the library, check out the Databricks [blog post](https://databricks.com/blog/2017/06/06/databricks-vision-simplify-large-scale-deep-learning.html). This notebook parallels the [Deep Learning Pipelines README](https://github.com/databricks/spark-deep-learning), detailing usage examples with additional tips for getting started with the library on Databricks.

## Cluster set-up

Deep Learning Pipelines is available as a Spark Package. To use it on your cluster, create a new library with the Source option "Maven Coordinate", using "Search Spark Packages and Maven Central" to find "spark-deep-learning". Then [attach the library to a cluster](https://docs.databricks.com/user-guide/libraries.html). **Note**: 
* This notebook works with spark-deep-learning release `0.1.0-spark2.1-s_2.11`. Future releases may introduce breaking changes. Please refer to the project's [github page](https://github.com/databricks/spark-deep-learning) for the latest examples and docs.
* To run this notebook, also create and attach the following libraries via PyPI: tensorflow, keras, h5py.

Deep Learning Pipelines is compatible with Spark versions 2.0 or higher and works with any instance type (CPU or GPU).

## Quick User Guide

Deep Learning Pipelines provides a suite of tools around working with and processing images using deep learning. The tools can be categorized as
* **Working with images** natively in Spark DataFrames
* **Transfer learning**, a super quick way to leverage deep learning
* **Applying deep learning models at scale**, whether they are your own or known popular models, to image data to make predictions or transform them into features
* **Deploying models as SQL functions** to empower everyone by making deep learning available in SQL (coming soon)
* **Distributed hyper-parameter tuning** via Spark MLlib Pipelines (coming soon)

We'll cover each one with examples below.

Let us first get some images to work with in this notebook. We'll use the flowers dataset from the [TensorFlow retraining tutorial](https://www.tensorflow.org/tutorials/image_retraining).

In [0]:
%sh 
curl -O http://download.tensorflow.org/example_images/flower_photos.tgz
tar xzf flower_photos.tgz

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 12 218M 12 28.0M 0 0 24.0M 0 0:00:09 0:00:01 0:00:08 24.0M
100 218M 100 218M 0 0 114M 0 0:00:01 0:00:01 --:--:-- 114M
tar: flower_photos/roses/14810868100_87eb739f26_m.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/1446090416_f0cad5fde4.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/15319767030_e6c5602a77_m.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/15032112248_30c5284e54_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/7211616670_2d49ecb3a5_m.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/15674450867_0ced942941_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/17158274118_00ec99a23c.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/14019883858_e5d2a0ec10_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/8035908422_87220425d2_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/14747962886_2bff6bb323_m.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/4356781875_92c5cd93c0.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/8524505546_b242bd4928_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/9406573080_60eab9278e_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/6039330368_c30ed224c4_m.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/14414100710_753a36fce9.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/3292434691_392071d702_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/5273722065_c85d8543c2_m.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/2215318403_06eb99176a.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/172311368_49412f881b.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/15255964454_0a64eb67fa.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/4504220673_af754fcb40_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/505517255_cfbb6f6394.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/6347846935_51e3dc2481_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/2273917656_6d6c038283.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/8775267816_726ddc6d92_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/5206847130_ee4bf0e4de_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/5777669976_a205f61e5b.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/14001990976_bd2da42dbc.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/4713531680_1110a2fa07_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/14154164774_3b39d36778.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/494803274_f84f21d53a.jpg: Cannot change ownership to uid 270850, gid 5000: In

In [0]:
display(dbutils.fs.ls('file:/databricks/driver/flower_photos'))

path,name,size
file:/databricks/driver/flower_photos/daisy/,daisy/,32768
file:/databricks/driver/flower_photos/dandelion/,dandelion/,49152
file:/databricks/driver/flower_photos/sunflowers/,sunflowers/,36864
file:/databricks/driver/flower_photos/LICENSE.txt,LICENSE.txt,418049
file:/databricks/driver/flower_photos/tulips/,tulips/,40960
file:/databricks/driver/flower_photos/roses/,roses/,36864


In [0]:
# The 'file:/...' directory will be cleared out upon cluster termination. That doesn't matter for this example notebook, but in most cases we'd want to store the images in a more permanent place. Let's move the files to dbfs so we can see how to work with it in the use cases below.
img_dir = '/tmp/flower_photos'
dbutils.fs.mkdirs(img_dir)
dbutils.fs.cp('file:/databricks/driver/flower_photos/tulips', img_dir + "/tulips", recurse=True)
dbutils.fs.cp('file:/databricks/driver/flower_photos/daisy', img_dir + "/daisy", recurse=True)
dbutils.fs.cp('file:/databricks/driver/flower_photos/LICENSE.txt', img_dir)
display(dbutils.fs.ls(img_dir))

path,name,size
dbfs:/tmp/flower_photos/LICENSE.txt,LICENSE.txt,418049
dbfs:/tmp/flower_photos/daisy/,daisy/,0
dbfs:/tmp/flower_photos/tulips/,tulips/,0


In [0]:
# Let's create a small sample set of images for quick demonstrations.
sample_img_dir = img_dir + "/sample"
dbutils.fs.mkdirs(sample_img_dir)
files = dbutils.fs.ls(img_dir + "/tulips")[0:1] + dbutils.fs.ls(img_dir + "/daisy")[0:2]
for f in files:
  dbutils.fs.cp(f.path, sample_img_dir)
display(dbutils.fs.ls(sample_img_dir))

path,name,size
dbfs:/tmp/flower_photos/sample/100080576_f52e8ee070_n.jpg,100080576_f52e8ee070_n.jpg,26797
dbfs:/tmp/flower_photos/sample/100930342_92e8746431_n.jpg,100930342_92e8746431_n.jpg,26200
dbfs:/tmp/flower_photos/sample/10140303196_b88d3d6cec.jpg,10140303196_b88d3d6cec.jpg,117247


### Working with images in Spark

The first step to applying deep learning on images is the ability to load the images. Deep Learning Pipelines includes utility functions that can load millions of images into a Spark DataFrame and decode them automatically in a distributed fashion, allowing manipulationg at scale.

In [0]:
from sparkdl import readImages
image_df = readImages(sample_img_dir)

/databricks/python/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
 from ._conv import register_converters as _register_converters
Using TensorFlow backend.

The resulting DataFrame contains a string column named "filePath" containing the path to each image file, and a image struct ("`SpImage`") column called "image" containing the decoded image data.   

In [0]:
display(image_df)

filePath,image
dbfs:/tmp/flower_photos/sample/100080576_f52e8ee070_n.jpg,"List(RGB, 263, 320, 3, ...)"
dbfs:/tmp/flower_photos/sample/100930342_92e8746431_n.jpg,"List(RGB, 209, 320, 3, ...)"


### Transfer learning
Deep Learning Pipelines provides utilities to perform transfer learning on images, which is one of the fastest (code and run-time -wise) ways to start using deep learning. Using Deep Learning Pipelines, it can be done in just several lines of code.

In [0]:
# Create training & test DataFrames for transfer learning - this piece of code is longer than transfer learning itself below!
from sparkdl import readImages
from pyspark.sql.functions import lit

tulips_df = readImages(img_dir + "/tulips").withColumn("label", lit(1))
daisy_df = readImages(img_dir + "/daisy").withColumn("label", lit(0))
tulips_train, tulips_test, _ = tulips_df.randomSplit([0.05, 0.05, 0.9])  # use larger training sets (e.g. [0.6, 0.4] for non-community edition clusters)
daisy_train, daisy_test, _ = daisy_df.randomSplit([0.05, 0.05, 0.9])     # use larger training sets (e.g. [0.6, 0.4] for non-community edition clusters)
train_df = tulips_train.unionAll(daisy_train)
test_df = tulips_test.unionAll(daisy_test)

# Under the hood, each of the partitions is fully loaded in memory, which may be expensive.
# This ensure that each of the paritions has a small size.
train_df = train_df.repartition(100)
test_df = test_df.repartition(100)

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer 

featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")
lr = LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label")
p = Pipeline(stages=[featurizer, lr])

p_model = p.fit(train_df)

Downloading data from https://github.com/fchollet/deep-learning-models/releases/download/v0.5/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

 16384/87910968 [..............................] - ETA: 0s
 24576/87910968 [..............................] - ETA: 7:06
 57344/87910968 [..............................] - ETA: 5:49
 73728/87910968 [..............................] - ETA: 6:31
 106496/87910968 [..............................] - ETA: 5:48
 139264/87910968 [..............................] - ETA: 5:27
 196608/87910968 [..............................] - ETA: 4:36
 262144/87910968 [..............................] - ETA: 4:01
 352256/87910968 [..............................] - ETA: 3:26
 458752/87910968 [..............................] - ETA: 2:59
 557056/87910968 [..............................] - ETA: 2:45
 696320/87910968 [..............................] - ETA: 2:26
 860160/87910968 [..............................] - ETA: 2:10
 1032192/87910968 [..............................] - ETA: 1:58
 1220608/87910968 [..............................] - ETA: 1:49
 1433600/87910968 [..............................] - ETA: 1:40
 1671168/87910968 [..............................] - ETA: 1:31
 1933312/87910968 [..............................] - ETA: 1:22
 2179072/87910968 [..............................] - ETA: 1:16
 2269184/87910968 [..............................] - ETA: 1:15
 2580480/87910968 [..............................] - ETA: 1:09
 2875392/87910968 [..............................] - ETA: 1:04
 2998272/87910968 [>.............................] - ETA: 1:03
 3366912/87910968 [>.............................] - ETA: 58s 
 3694592/87910968 [>.............................] - ETA: 54s
 3866624/87910968 [>.............................] - ETA: 53s
 4341760/87910968 [>.............................] - ETA: 49s
 4759552/87910968 [>.............................] - ETA: 45s
 4964352/87910968 [>.............................] - ETA: 44s
 5545984/87910968 [>.............................] - ETA: 40s
 6037504/87910968 [=>............................] - ETA: 38s
 6299648/87910968 [=>............................] - ETA: 37s
 6971392/87910968 [=>............................] - ETA: 34s
 7561216/87910968 [=>............................] - ETA: 32s
 7872512/87910968 [=>............................] - ETA: 31s
 8675328/87910968 [=>............................] - ETA: 29s
 9379840/87910968 [==>...........................] - ETA: 27s
 9756672/87910968 [==>...........................] - ETA: 26s
10706944/87910968 [==>...........................] - ETA: 24s
11526144/87910968 [==>...........................] - ETA: 23s
11984896/87910968 [===>..........................] - ETA: 22s
12853248/87910968 [===>..........................] - ETA: 21s
14475264/87910968 [===>..........................] - ETA: 19s
15376384/87910968 [====>.........................] - ETA: 18s
15638528/87910968 [====>.........................] - ETA: 18s
15818752/87910968 [====>.........................] - ETA: 18s
16195584/87910968 [====>.........................] - ETA: 18s
16605184/87910968 [====>.........................] - ETA: 17s
16867328/87910968 [====>.........................] - ETA: 17s
17391616/87910968 [====>.........................] - ETA: 17s
17555456/87910968 [====>.........................] - ETA: 17s
18767872/87910968 [=====>........................] - ETA: 16s
19128320/87910968 [=====>........................] - ETA: 16s
19259392/87910968 [=====>........................] - ETA: 16s
19603456/87910968 [=====>........................] - ETA: 16s
19931136/87910968 [=====>........................] - ETA: 16s
20144128/87910968 [=====>........................] - ETA: 16s
20635648/87910968 [======>.......................] - ETA: 16s
21094400/87910968 [======>.......................] - ETA: 15s
21372928/87910968 [======>.......................] - ETA: 15s
21897216/87910968 [======>.......................] - ETA: 15s
22388736/87910968 [======>.......................] - ETA: 15s

Note: the training step may take a while on Community Edition - try making a smaller training set in that case.

Let's see how well the model does:

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

tested_df = p_model.transform(test_df)
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(tested_df.select("prediction", "label"))))

INFO:tensorflow:Froze 376 variables.
Converted 376 variables to const ops.
INFO:tensorflow:Froze 0 variables.
Converted 0 variables to const ops.
Test set accuracy = 0.971014492754

Not bad for a first try with zero tuning! Furthermore, we can look at where we are making mistakes:

In [0]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import expr
def _p1(v):
  return float(v.array[1])
p1 = udf(_p1, DoubleType())

df = tested_df.withColumn("p_1", p1(tested_df.probability))
wrong_df = df.orderBy(expr("abs(p_1 - label)"), ascending=False)
display(wrong_df.select("filePath", "p_1", "label").limit(10))

filePath,p_1,label
dbfs:/tmp/flower_photos/daisy/9345273630_af3550031d.jpg,0.804202936186138,0
dbfs:/tmp/flower_photos/daisy/530738000_4df7e4786b.jpg,0.6165413243876156,0
dbfs:/tmp/flower_photos/tulips/113902743_8f537f769b_n.jpg,0.5153939149058596,1
dbfs:/tmp/flower_photos/tulips/14053292975_fdc1093571_n.jpg,0.5376996385350966,1
dbfs:/tmp/flower_photos/tulips/2294116183_a30d2aa2c1_m.jpg,0.6245144314249896,1
dbfs:/tmp/flower_photos/daisy/15813862117_dedcd1c56f_m.jpg,0.3507493964314712,0
dbfs:/tmp/flower_photos/tulips/14674071872_2df55466d5_m.jpg,0.6689220612741658,1
dbfs:/tmp/flower_photos/tulips/4591323356_030d8b6967_m.jpg,0.6965201230884019,1
dbfs:/tmp/flower_photos/tulips/4590703575_6371c0a186_n.jpg,0.7213367030778205,1
dbfs:/tmp/flower_photos/tulips/7166567320_0a2beb6d42.jpg,0.7230501393191048,1


### Applying deep learning models at scale
Spark DataFrames are a natural construct for applying deep learning models to a large-scale dataset. Deep Learning Pipelines provides a set of (Spark MLlib) Transformers for applying TensorFlow Graphs and TensorFlow-backed Keras Models at scale. In addition, popular images models can be applied out of the box, without requiring any TensorFlow or Keras code. The Transformers, backed by the Tensorframes library, efficiently handle the distribution of models and data to Spark workers.

#### Applying popular image models
There are many well-known deep learning models for images. If the task at hand is very similar to what the models provide (e.g. object recognition with ImageNet classes), or for pure exploration, one can use the Transformer `DeepImagePredictor` by simply specifying the model name.  

In [0]:
from sparkdl import readImages, DeepImagePredictor

image_df = readImages(sample_img_dir)

predictor = DeepImagePredictor(inputCol="image", outputCol="predicted_labels", modelName="InceptionV3", decodePredictions=True, topK=10)
predictions_df = predictor.transform(image_df)

display(predictions_df.select("filePath", "predicted_labels"))

filePath,predicted_labels
dbfs:/tmp/flower_photos/sample/100080576_f52e8ee070_n.jpg,"List(List(n11939491, daisy, 0.8805494), List(n02219486, ant, 0.0020712544), List(n02206856, bee, 8.4249553E-4), List(n02165456, ladybug, 6.75952E-4), List(n03691459, loudspeaker, 6.3085736E-4), List(n02190166, fly, 6.220306E-4), List(n02281406, sulphur_butterfly, 6.154293E-4), List(n07930864, cup, 5.5486656E-4), List(n02112018, Pomeranian, 4.993835E-4), List(n07745940, strawberry, 4.8750438E-4))"
dbfs:/tmp/flower_photos/sample/100930342_92e8746431_n.jpg,"List(List(n03930313, picket_fence, 0.18473865), List(n11939491, daisy, 0.10779827), List(n03991062, pot, 0.068061665), List(n02206856, bee, 0.05250212), List(n02280649, cabbage_butterfly, 0.020910017), List(n12620546, hip, 0.014124007), List(n03457902, greenhouse, 0.012763838), List(n12057211, yellow_lady's_slipper, 0.012688815), List(n13133613, ear, 0.012625335), List(n02281406, sulphur_butterfly, 0.012392867))"
dbfs:/tmp/flower_photos/sample/10140303196_b88d3d6cec.jpg,"List(List(n11939491, daisy, 0.9535933), List(n02219486, ant, 5.922901E-4), List(n02206856, bee, 5.66493E-4), List(n02165456, ladybug, 4.277229E-4), List(n02190166, fly, 3.9971404E-4), List(n02281406, sulphur_butterfly, 3.0156912E-4), List(n02112018, Pomeranian, 2.741234E-4), List(n02177972, weevil, 2.677667E-4), List(n01795545, black_grouse, 2.5603166E-4), List(n04592741, wing, 2.2698511E-4))"


Notice that the `predicted_labels` column shows "daisy" as a high probability class for all sample flowers using this base model. However, as can be seen from the differences in the probability values, the neural network has the information to discern the two flower types. Hence our transfer learning example above was able to properly learn the differences between daisies and tulips starting from the base model.

In [0]:
df = p_model.transform(image_df)
display(df.select("filePath", (1-p1(df.probability)).alias("p_daisy")))

filePath,p_daisy
dbfs:/tmp/flower_photos/sample/100080576_f52e8ee070_n.jpg,0.968875532556084
dbfs:/tmp/flower_photos/sample/100930342_92e8746431_n.jpg,0.13976502414604564
dbfs:/tmp/flower_photos/sample/10140303196_b88d3d6cec.jpg,0.9786112803439984


#### For TensorFlow users
Deep Learning Pipelines provides a MLlib Transformer that will apply the given TensorFlow Graph to a DataFrame containing a column of images (e.g. loaded using the utilities described in the previous section). Here is a very simple example of how a TensorFlow Graph can be used with the Transformer. In practice, the TensorFlow Graph will likely be restored from files before calling `TFImageTransformer`.

In [0]:
from sparkdl import readImages, TFImageTransformer
from sparkdl.transformers import utils
import tensorflow as tf

image_df = readImages(sample_img_dir)

g = tf.Graph()
with g.as_default():
    image_arr = utils.imageInputPlaceholder()
    resized_images = tf.image.resize_images(image_arr, (299, 299))
    # the following step is not necessary for this graph, but can be for graphs with variables, etc
    frozen_graph = utils.stripAndFreezeGraph(g.as_graph_def(add_shapes=True), tf.Session(graph=g), [resized_images])
      
transformer = TFImageTransformer(inputCol="image", outputCol="transformed_img", graph=frozen_graph,
                                 inputTensor=image_arr, outputTensor=resized_images,
                                 outputMode="image")
tf_trans_df = transformer.transform(image_df)

INFO:tensorflow:Froze 0 variables.
Converted 0 variables to const ops.
INFO:tensorflow:Froze 0 variables.
Converted 0 variables to const ops.

#### For Keras users
For applying Keras models in a distributed manner using Spark, [`KerasImageFileTransformer`](link_here) works on TensorFlow-backed Keras models. It 
* Internally creates a DataFrame containing a column of images by applying the user-specified image loading and processing function to the input DataFrame containing a column of image URIs
* Loads a Keras model from the given model file path 
* Applies the model to the image DataFrame

The difference in the API from `TFImageTransformer` above stems from the fact that usual Keras workflows have very specific ways to load and resize images that are not part of the TensorFlow Graph.

To use the transformer, we first need to have a Keras model stored as a file. For this notebook we'll just save the Keras built-in InceptionV3 model instead of training one.

In [0]:
from keras.applications import InceptionV3

model = InceptionV3(weights="imagenet")
model.save('/tmp/model-full.h5')  # saves to the local filesystem
# move to a permanent place for future use
dbfs_model_path = 'dbfs:/models/model-full.h5'
dbutils.fs.cp('file:/tmp/model-full.h5', dbfs_model_path)  

Out[ 14 ]: True

Now on the prediction side:

In [0]:
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array, load_img
import numpy as np
from pyspark.sql.types import StringType
from sparkdl import KerasImageFileTransformer

def loadAndPreprocessKerasInceptionV3(uri):
  # this is a typical way to load and prep images in keras
  image = img_to_array(load_img(uri, target_size=(299, 299)))  # image dimensions for InceptionV3
  image = np.expand_dims(image, axis=0)
  return preprocess_input(image)

dbutils.fs.cp(dbfs_model_path, 'file:/tmp/model-full-tmp.h5')
transformer = KerasImageFileTransformer(inputCol="uri", outputCol="predictions",
                                        modelFile='/tmp/model-full-tmp.h5',  # local file path for model
                                        imageLoader=loadAndPreprocessKerasInceptionV3,
                                        outputMode="vector")

files = ["/dbfs" + str(f.path)[5:] for f in dbutils.fs.ls(sample_img_dir)]  # make "local" file paths for images
uri_df = sqlContext.createDataFrame(files, StringType()).toDF("uri")

keras_pred_df = transformer.transform(uri_df)

/databricks/python/local/lib/python2.7/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
 warnings.warn('No training configuration found in save file: '
INFO:tensorflow:Froze 378 variables.
Converted 378 variables to const ops.
INFO:tensorflow:Froze 0 variables.
Converted 0 variables to const ops.

In [0]:
display(keras_pred_df.select("uri", "predictions"))

uri predictions /dbfs/tmp/flower_photos/sample/100080576_f52e8ee070_n.jpg List(1, 1000, List(), List(7.469155389117077E-5, 7.630845357198268E-5, 1.935783657245338E-4, 1.2283110118005425E-4, 7.464548980351537E-5, 1.4129254850558937E-4, 7.751797238597646E-5, 7.233556971186772E-5, 8.64552057464607E-5, 1.5517773863393813E-4, 1.0027247481048107E-4, 1.113280959543772E-4, 1.1478483065729961E-4, 1.7278207815252244E-4, 1.1247883958276361E-4, 8.04331575636752E-5, 9.47006992646493E-5, 5.863575279363431E-5, 7.717055996181443E-5, 2.1305341215338558E-4, 6.69631699565798E-5, 1.534194452688098E-4, 2.261035260744393E-4, 1.1289042595308274E-4, 5.5337921367026865E-5, 5.628731014439836E-5, 5.129736746312119E-5, 7.584730337839574E-5, 9.452384983887896E-5, 8.727449312573299E-5, 1.1334264854667708E-4, 1.059042988345027E-4, 7.200918480521068E-5, 7.737254782114178E-5, 7.67828751122579E-5, 1.2126631918363273E-4, 2.01993461814709E-4, 7.560313679277897E-5, 1.2823846191167831E-4, 6.0258091252762824E-5, 5.496805169968866E-5, 9.389923798153177E-5, 7.733707025181502E-5, 8.161546429619193E-5, 1.2234253517817706E-4, 7.5049429142382E-5, 1.29581312648952E-4, 7.247367466334254E-5, 1.8120465392712504E-4, 6.954235141165555E-5, 1.1758445180021226E-4, 4.686410466092639E-5, 1.2563297059386969E-4, 1.6615306958556175E-4, 1.2340655666776001E-4, 1.496839540777728E-4, 2.7740723453462124E-4, 1.5808192256372422E-4, 2.0232873794157058E-4, 6.950760871404782E-5, 5.229574890108779E-5, 1.1755765444831923E-4, 4.45148725702893E-5, 7.99452536739409E-5, 1.8889657803811133E-4, 3.401203503017314E-5, 8.955553494160995E-5, 1.8168744281865656E-4, 2.937536046374589E-4, 6.833047518739477E-5, 1.8812730559147894E-4, 1.025923338602297E-4, 6.0771722928620875E-5, 1.0247069440083578E-4, 1.7255237617064267E-4, 1.8552533583715558E-4, 6.807115278206766E-5, 5.6424418289680034E-5, 3.023524477612227E-4, 1.2683210661634803E-4, 3.69126966688782E-4, 8.583100861869752E-5, 7.208751048892736E-5, 6.943214975763112E-5, 8.355187310371548E-5, 5.4272506531560794E-5, 3.810115595115349E-5, 9.354305075248703E-5, 3.388098775758408E-5, 9.22415274544619E-5, 6.060560917831026E-5, 4.135833660257049E-5, 5.8230180002283305E-5, 6.604034570045769E-5, 1.2170913396403193E-4, 7.806793291820213E-5, 5.3572373872157186E-5, 9.256546763936058E-5, 1.0608392040012404E-4, 2.997385454364121E-4, 1.949902216438204E-4, 6.731488247169182E-5, 7.421299233101308E-5, 9.00381855899468E-5, 3.844149978249334E-5, 1.084081013686955E-4, 1.3273447984829545E-4, 5.66693015571218E-5, 3.406690084375441E-4, 6.551347905769944E-5, 2.6872579837800004E-5, 6.049600779078901E-5, 5.33117818122264E-5, 1.9492459250614047E-4, 1.1158745473949239E-4, 3.259751247242093E-5, 3.5239623684901744E-5, 6.693756586173549E-5, 8.302548667415977E-5, 1.5900322978268377E-5, 7.125772390281782E-5, 4.905109744868241E-5, 5.707270611310378E-5, 5.849511580890976E-5, 1.5325989807024598E-4, 3.94623311876785E-5, 3.623664451879449E-5, 1.5781755791977048E-4, 1.682561414781958E-4, 9.739832603372633E-5, 2.0636458066292107E-4, 7.979642396094278E-5, 1.456497993785888E-4, 1.1108788021374494E-4, 2.5753662339411676E-4, 1.8849708430934697E-4, 9.723544644657522E-5, 1.6008650709409267E-4, 1.0348537034587935E-4, 9.095914720091969E-5, 4.953151074005291E-5, 6.714298797305673E-5, 8.824298129184172E-5, 1.0243435099255294E-4, 7.812908006599173E-5, 9.871667134575546E-5, 1.4981561980675906E-4, 1.1993070074822754E-4, 4.343882028479129E-4, 1.014280496747233E-4, 6.210516585269943E-5, 1.0073638259200379E-4, 1.736195117700845E-4, 3.199405618943274E-4, 8.302176865981892E-5, 1.697380794212222E-4, 9.130348189501092E-5, 1.0218008537776768E-4, 1.5227329276967794E-4, 1.010148916975595E-4, 1.1895847273990512E-4, 5.516756937140599E-5, 1.3864283391740173E-4, 9.551119001116604E-5, 3.256103445892222E-5, 1.379012392135337E-4, 1.4380502398125827E-4, 1.380142493871972E-4, 8.52421362651512E-5, 7.916924369055778E-5, 4.691562207881361E-5, 1.6043435607571155E-4, 1.9104560487903655E-4, 7.345191988861188E-5, 8.920137042878196E-5, 1.08

#### Clean up data generated for this notebook

In [0]:
dbutils.fs.rm(img_dir, recurse=True)
dbutils.fs.rm(dbfs_model_path)

Out[ 17 ]: True

### Resources
* See the Databricks [blog post](https://databricks.com/blog/2017/06/06/databricks-vision-simplify-large-scale-deep-learning.html) announcing Deep Learning Pipelines for a high-level overview and more in-depth discussion of some of the concepts here.
* Check out the [Deep Learning Pipelines github page](https://github.com/databricks/spark-deep-learning).
* Learn more about [deep learning on Databricks](https://docs.databricks.com/applications/deep-learning/index.html).